In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
import random
import datetime

load_dotenv()

openai_client = OpenAI()

In [2]:
DATA_PATH = "KareCoder-main/CodeF dataset/CodeF/"
PRE2021 = DATA_PATH + "CodeF pre2021-9/"
POST2021 = DATA_PATH + "CodeF post2021-9/"

In [3]:
def load_problem(id, pre2021=True):
    # add 0 prefix to id to make it 4 digits
    id = str(id).zfill(4)

    if pre2021:
        path = PRE2021 + id + "/"

        input_output_path = os.path.join(path, "input_output.json")
        metadata_path = os.path.join(path, "metadata.json")
        question_path = os.path.join(path, "question.txt")
        solution_path = os.path.join(path, "solution.json")

        with open(input_output_path) as f:
            input_output = json.load(f)

        with open(metadata_path) as f:
            metadata = json.load(f)

        with open(question_path) as f:
            question = f.read()

        with open(solution_path) as f:
            solution = json.load(f)
    else:
        path = POST2021 + id + "/"

        input_output_path = os.path.join(path, "input_output.json")
        metadata_path = os.path.join(path, "metadata.json")
        question_path = os.path.join(path, "question.txt")
        solution_path = os.path.join(path, "solution.json")

        with open(input_output_path) as f:
            input_output = json.load(f)

        with open(metadata_path) as f:
            metadata = json.load(f)

        with open(question_path) as f:
            question = f.read()

        with open(solution_path) as f:
            solution = json.load(f)

    return {
        "input_output": input_output,
        "metadata": metadata,
        "question": question,
        "solution": solution
    }

In [4]:
KNOWLEDGE_LIBRARY = "KareCoder-main/Knowledge Libraries/"

In [5]:
def load_knowledge_library():
    knowledge_library = {}

    with open(KNOWLEDGE_LIBRARY + "Knowledge Description.json") as f:
        knowledge_library["description"] = json.load(f)
    
    with open(KNOWLEDGE_LIBRARY + "Knowledge Pseudo-Code.json") as f:
        knowledge_library["pseudo_code"] = json.load(f)

    with open(KNOWLEDGE_LIBRARY + "Knowledge Step of Pseudo-Code.json") as f:
        knowledge_library["steps"] = json.load(f)

    return knowledge_library

In [6]:
prompt_engineer_role = """You are a prompt engineer for LLM. Your job is to generate prompts based on the example given by user and output the prompt."""

prompt_engineer_prefix = """Suppose you are a prompt engineer, accountable for comprehending the problem and learning as well as reviewing the algorithms and data structures knowledge pertinent to resolving the issue, subsequently generating a problem-solving prompt.

Below is an example of what you are expected to do.

------------------Begin------------------

# Problem:

Each test contains multiple test cases. The first line contains the number of test cases $t$ ($1 \leq t \leq 10^4$). Description of the test cases follows.

The first line of each test case contains two integers $n$ and $m$ ($1 \le n, m \le 1000$)  — the size of the grid.

Each of the following $n$ lines contains $m$ integers. The $j$-th integer on the $i$-th line is $a_{ij}$ ($a_{ij} = 1$ or $-1$)  — the element in the cell $(i, j)$.

It is guaranteed that the sum of $n\cdot m$ over all test cases does not exceed $10^6$.


-----Output-----

For each test case, print "YES" if there exists a path from the top left to the bottom right that adds up to $0$, and "NO" otherwise. You can output each letter in any case.


-----Examples-----

Input
5
1 1
1
1 2
1 -1
1 4
1 -1 1 -1
3 4
1 -1 -1 -1
-1 1 1 -1
1 1 1 -1
3 4
1 -1 1 1
-1 1 -1 1
1 -1 1 1
Output
NO
YES
YES
YES
NO

-----Note-----

One possible path for the fourth test case is given in the picture in the statement.

# Knowledge Description:

"DP":"DP, or dynamic programming, is a technique used to solve complex problems by breaking them down into smaller, simpler sub-problems. It involves solving each sub-problem only once and storing the solutions in a table for future use. DP is used in optimization problems where the goal is to find the optimal solution among many possible solutions. It is often more efficient than brute force or other methods, but requires careful analysis and design of the sub-problems. DP is commonly used in fields such as computer science, engineering, and finance."

"Data Structures":"Data structures are arrangements of data in a computer's memory that enable efficient access, storage, and manipulation of information. They provide a means for organizing and managing data, and are essential for efficient algorithm design. Examples of data structures include arrays, linked lists, stacks, queues, trees, and graphs. Each data structure has its own advantages and disadvantages, and selecting the appropriate structure depends on the nature of the data and the operations to be performed on it."

"Brute Force":"Brute force is a straightforward approach to problem-solving that exhaustively searches all possible solutions. It involves systematically testing each possible solution until the correct one is found. This approach is often used when the problem size is small, or when other methods are not feasible. Brute force can be computationally expensive and impractical for large-scale problems, but it is guaranteed to find the optimal solution, if one exists. It is commonly used in algorithm design for testing and benchmarking."

# Knowledge-Aware Prompt:

1.Read t, the number of test cases.

2.Loop t times to read each test cases.

3.Read n and m, the dimensions of the matrix.

4.Read the elements of the n x m matrix and store them in a 2D list a.

5.Check if (n + m - 1) is odd. If it is, print "NO" and continue to the next test case.

6.Create a list dp of length m + 1, initialized with all zeros.

7.Loop n times, allet dp[0] to 1 if i is 0, otherwise keep it 0.

8.Loop m times, compute the value of dp[j + 1] based on dp[j], dp[j + 1] and the element at a[i][j]. The bit shift operation is used here to handle the state transition.

9.Check the (n+m-1)//2 bit of dp[-1]. If it is 1, print "YES"; otherwise, print "NO".

10.End all loop.

-------------------End-------------------

Now, with a new question and knowledge description, write the knowledge-aware prompt.
"""

<>:3: SyntaxWarning: invalid escape sequence '\l'
<>:3: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_3495/764254301.py:3: SyntaxWarning: invalid escape sequence '\l'
  prompt_engineer_prefix = """Suppose you are a prompt engineer, accountable for comprehending the problem and learning as well as reviewing the algorithms and data structures knowledge pertinent to resolving the issue, subsequently generating a problem-solving prompt.


In [7]:
coder_role = """You are a python coder. Your job is to write python code based on example given by user."""

coder_prefix = """Now, suppose that you are a coder, tasked with perusing the problem and implementing this solution in a programming language, guided by the prompt provided by the prompt engineer.

Below is an example of what you are expected to do.

------------------Begin------------------

# Problem:

Each test contains multiple test cases. The first line contains the number of test cases $t$ ($1 \leq t \leq 10^4$). Description of the test cases follows.

The first line of each test case contains two integers $n$ and $m$ ($1 \le n, m \le 1000$)  — the size of the grid.

Each of the following $n$ lines contains $m$ integers. The $j$-th integer on the $i$-th line is $a_{ij}$ ($a_{ij} = 1$ or $-1$)  — the element in the cell $(i, j)$.

It is guaranteed that the sum of $n\cdot m$ over all test cases does not exceed $10^6$.


-----Output-----

For each test case, print "YES" if there exists a path from the top left to the bottom right that adds up to $0$, and "NO" otherwise. You can output each letter in any case.


-----Examples-----

Input
5
1 1
1
1 2
1 -1
1 4
1 -1 1 -1
3 4
1 -1 -1 -1
-1 1 1 -1
1 1 1 -1
3 4
1 -1 1 1
-1 1 -1 1
1 -1 1 1
Output
NO
YES
YES
YES
NO

-----Note-----

One possible path for the fourth test case is given in the picture in the statement.

# Knowledge-Aware Prompt:

1.Read t, the number of test cases.

2.Loop t times to read each test cases.

3.Read n and m, the dimensions of the matrix.

4.Read the elements of the n x m matrix and store them in a 2D list a.

5.Check if (n + m - 1) is odd. If it is, print "NO" and continue to the next test case.

6.Create a list dp of length m + 1, initialized with all zeros.

7.Loop n times, allet dp[0] to 1 if i is 0, otherwise keep it 0.

8.Loop m times, compute the value of dp[j + 1] based on dp[j], dp[j + 1] and the element at a[i][j]. The bit shift operation is used here to handle the state transition.

9.Check the (n+m-1)//2 bit of dp[-1]. If it is 1, print "YES"; otherwise, print "NO".

10.End all loop.

# Solution Code:

```python  
for case in range(int(input())):
    n, m = map(int, input().split())
    a = [list(map(int, input().split())) for _ in range(n)]
    if (n + m - 1) % 2:
        print("NO")
        continue
    dp = [0] * (m + 1)
    for i in range(n):
        dp[0] = int(not i)
        for j in range(m):
            dp[j + 1] = (dp[j] | dp[j + 1]) << (a[i][j] == -1)
    if dp[-1] & (1 << ((n + m - 1) // 2)):
        print("YES")
    else:
        print("NO")
```

-------------------End-------------------

Now, with a new question and knowledge-aware prompt, write the solution code.
"""

<>:3: SyntaxWarning: invalid escape sequence '\l'
<>:3: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_3495/1548791745.py:3: SyntaxWarning: invalid escape sequence '\l'
  coder_prefix = """Now, suppose that you are a coder, tasked with perusing the problem and implementing this solution in a programming language, guided by the prompt provided by the prompt engineer.


In [8]:
def generate_prompt_engineer_prompt(id, pre2021=True):
    knowledge_library = load_knowledge_library()
    problem = load_problem(id, pre2021=pre2021)

    tags = problem["metadata"]["tag"]
    knowledge_description = ""
    for tag in tags:
        knowledge_description += tag + ": " + knowledge_library["description"][tag] + "\n\n"

    prompt =    (prompt_engineer_prefix + 
                "\n# Problem:\n\n" +
                problem["question"] + 
                "\n\n# Knowledge Description:\n\n" + 
                knowledge_description
                )

    return prompt

# engineer_prompt = generate_prompt_engineer_prompt(13, pre2021=False)
# print(engineer_prompt)

In [9]:
def generate_coder_prompt(id, knowledge_aware_prompt, pre2021=True):
    problem = load_problem(id, pre2021=pre2021)

    prompt =    (coder_prefix + 
                "\n# Problem:\n\n" +
                problem["question"] + 
                "\n\n" + 
                knowledge_aware_prompt
                )

    return prompt

# dummy_prompt = "1.Read t, the number of test cases."
# coder_prompt = generate_coder_prompt(13, dummy_prompt, pre2021=False)
# print(coder_prompt)

In [10]:
def prompt_engineer(client, id, pre2021=True):
    prompt_engineer_input = generate_prompt_engineer_prompt(id, pre2021=pre2021)

    completion = client.chat.completions.create(
        model = "gpt-3.5-turbo-0125",
        messages = [
            {"role": "system", "content": prompt_engineer_role},
            {"role": "user", "content": prompt_engineer_input}
        ]
    )

    return completion.choices[0].message.content

def coder(client, id, knowledge_aware_prompt, pre2021=True):
    coder_input = generate_coder_prompt(id, knowledge_aware_prompt, pre2021=pre2021)

    completion = client.chat.completions.create(
        model = "gpt-3.5-turbo-0125",
        messages = [
            {"role": "system", "content": coder_role},
            {"role": "user", "content": coder_input}
        ]
    )

    return completion.choices[0].message.content

In [11]:
def generate_solution(client, id, timestamp, pre2021=True):
    path = "outputs/run_" + str(timestamp) + "/"
    os.makedirs(path, exist_ok=True)

    if pre2021:
        path += "pre2021-9/"
        os.makedirs(path, exist_ok=True)
    else:
        path += "post2021-9/"
        os.makedirs(path, exist_ok=True)

    os.makedirs(path + "knowledge_aware_prompts/", exist_ok=True)
    os.makedirs(path + "solution_codes/", exist_ok=True)

    knowledge_aware_prompt = prompt_engineer(client, id, pre2021=pre2021)
    with open(path + "knowledge_aware_prompts/" + str(id) + "_prompt.txt", "w") as f:
        f.write(knowledge_aware_prompt)
    solution_code = coder(client, id, knowledge_aware_prompt, pre2021=pre2021)
    # strip the ```python and ``` from the solution code, if there is any
    solution_code = solution_code.replace("```python", "").replace("```", "").strip()
    with open(path + "solution_codes/" + str(id) + "_solution.py", "w") as f:
        f.write(solution_code)

# generate_solution(openai_client, 13, pre2021=False)

In [12]:
# num_problems = 1

# random.seed(0)

# timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")

# random_problems = random.sample(range(0, 805), num_problems)

# for problem_id in random_problems:
#     generate_solution(openai_client, problem_id, timestamp, pre2021=True)

# random_problems = random.sample(range(0, 718), num_problems)

# for problem_id in random_problems:
#     generate_solution(openai_client, problem_id, timestamp, pre2021=False)

In [13]:
num_problems = 30

random.seed(42)

# selection subset of problems with difficulties matching the original paper (table 1)
# see KareCoder-main/CodeF dataset/Difficulty distributions of CodeF.xlsx
# simple: 800
# mdeium: 1300-1600
# hard: 1900-2500

dataset_ids = {
    "post2021": {
        "simple": range(0, 212),
        "medium": range(388, 527),
        "hard": range(597, 686)
    },
    "pre2021": {
        "simple": range(0, 150),
        "medium": range(333, 543),
        "hard": range(630, 754)
    }
}

timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")

# for pre2021 and post2021, generate num_problems of each difficulty
for dataset in dataset_ids:
    for difficulty in dataset_ids[dataset]:
        random_problems = random.sample(dataset_ids[dataset][difficulty], num_problems)

        for problem_id in random_problems:
            generate_solution(openai_client, problem_id, timestamp, pre2021=(dataset == "pre2021"))